<a href="https://colab.research.google.com/github/TPShipilova/Frameworks_LAB_4COURSE/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0%D1%8F%E2%84%961_%D0%9F%D0%A1%D0%A4%D0%98%D0%98_%D0%A8%D0%B8%D0%BF%D0%B8%D0%BB%D0%BE%D0%B2%D0%B0_406.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Начальные условия

**Классификация**

Набор данных: Loan Approval Classification Dataset.

Целевая переменная: loan_status (1 = одобрена, 0 = отклонена).

Обоснование: Задача предсказания одобрения кредита — классическая и крайне важная задача в менеджменте банков. Автоматизация этого процесса позволяет снизить субъективность, ускорить обработку заявок и минимизировать риски дефолта. Выбранный датасет синтетический, что снимает проблемы конфиденциальности, но при этом сохраняет реальные зависимости и проблемы (например, дисбаланс классов, смешанные типы данных).

**Регрессия**

Набор данных: Тот же Loan Approval Classification Dataset.

Целевая переменная: credit_score (кредитный скоринг).

Обоснование: Кредитный скоринг — ключевой показатель при оценке платёжеспособности клиента. Точное предсказание кредитного скоринга на основе финансовых и демографических данных позволяет банкам более тонко настраивать условия кредита (например, процентную ставку) и точнее оценивать риски. Использование одного датасета для двух задач демонстрирует универсальность данных.

**Метрики качества**

Для классификации (loan_status):

- F1-Score (F-мера): Основная метрика. Поскольку последствия ошибок "False Approved" (одобрить плохой кредит) и "False Rejected" (отклонить хороший кредит) могут быть дорогими, нам важна гармоничная средняя между точностью (Precision) и полнотой (Recall). Это особенно актуально при возможном дисбалансе классов.

- Accuracy (Точность): Дополнительная метрика. Полезна для общей оценки, но может вводить в заблуждение на несбалансированных данных.

- ROC-AUC: Показывает, насколько хорошо модель отделяет класс "одобренных" кредитов от "отклоненных". Хорошая обобщающая метрика.

Для регрессии (credit_score):

- RMSE (Root Mean Squared Error): Основная метрика. Показывает среднюю величину ошибки в единицах целевой переменной (в пунктах кредитного скоринга). Чувствительна к большим ошибкам, что важно для риск-менеджмента.

- MAE (Mean Absolute Error): Дополнительная метрика. Интерпретируется проще, чем RMSE ("средняя ошибка в пунктах").

- R² (Коэффициент детерминации): Показывает, какая доля дисперсии целевой переменной объясняется моделью.

# 2. Создание бейзлайна и оценка качества

In [2]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, mean_squared_error, mean_absolute_error, r2_score

In [4]:
# Загрузка данных
df = pd.read_csv('/content/loan_data.csv')

In [6]:
# Предварительный анализ данных
print(df.shape)
print(df.info())
print(df.isnull().sum())
df.describe()

(45000, 14)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45000 entries, 0 to 44999
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      45000 non-null  float64
 1   person_gender                   45000 non-null  object 
 2   person_education                45000 non-null  object 
 3   person_income                   45000 non-null  float64
 4   person_emp_exp                  45000 non-null  int64  
 5   person_home_ownership           45000 non-null  object 
 6   loan_amnt                       45000 non-null  float64
 7   loan_intent                     45000 non-null  object 
 8   loan_int_rate                   45000 non-null  float64
 9   loan_percent_income             45000 non-null  float64
 10  cb_person_cred_hist_length      45000 non-null  float64
 11  credit_score                    45000 non-null  int64  
 12  previous_loan_defaul

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status
count,45000.000000,4.500000e+04,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000,45000.000000
mean,27.764178,8.031905e+04,5.410333,9583.157556,11.006606,0.139725,5.867489,632.608756,0.222222
std,6.045108,8.042250e+04,6.063532,6314.886691,2.978808,0.087212,3.879702,50.435865,0.415744
min,20.000000,8.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,390.000000,0.000000
25%,24.000000,4.720400e+04,1.000000,5000.000000,8.590000,0.070000,3.000000,601.000000,0.000000
50%,26.000000,6.704800e+04,4.000000,8000.000000,11.010000,0.120000,4.000000,640.000000,0.000000
75%,30.000000,9.578925e+04,8.000000,12237.250000,12.990000,0.190000,8.000000,670.000000,0.000000
max,144.000000,7.200766e+06,125.000000,35000.000000,20.000000,0.660000,30.000000,850.000000,1.000000


Видим, что в датасете 14 колонок, среди признаков есть и категориальные (5 признаков), и числовые (7 признаков), что необходимо учесть при препроцессинге. Кроме того, пропусков в данных нет.

# Задача классификации





Для начала разделим датасет на признаки и таргет, а так же выделим обучающую и тестовую выборки для обучения модели.

In [ ]:
X_clf = df.drop('loan_status', axis=1)
y_clf = df['loan_status']


X_clf_train, X_clf_test, y_clf_train, y_clf_test = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42, stratify=y_clf
)

Определение числовых и категориальных столбцов

In [ ]:
numerical_features_clf = X_clf.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features_clf = X_clf.select_dtypes(include=['object']).columns.tolist()

Так как данные есть и категориальные, и численные, учтем это в препроцессинге.

Классификатор берем из обычной библиотеки sklearn.

In [ ]:
preprocessor_clf = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_clf),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_clf)
    ])

baseline_knn_clf = Pipeline(steps=[
    ('preprocessor', preprocessor_clf),
    ('classifier', KNeighborsClassifier())
])

Обучим библиотечную модель, сделаем предсказание и оценим результаты.

In [ ]:
baseline_knn_clf.fit(X_clf_train, y_clf_train)

y_clf_pred = baseline_knn_clf.predict(X_clf_test)
y_clf_pred_proba = baseline_knn_clf.predict_proba(X_clf_test)[:, 1] # для ROC-AUC

print("=== БЕЙЗЛАЙН - КЛАССИФИКАЦИЯ ===")
print(f"F1-Score: {f1_score(y_clf_test, y_clf_pred):.4f}")
print(f"Accuracy: {accuracy_score(y_clf_test, y_clf_pred):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_clf_test, y_clf_pred_proba):.4f}")

=== БЕЙЗЛАЙН - КЛАССИФИКАЦИЯ ===
F1-Score: 0.7541
Accuracy: 0.8956
ROC-AUC: 0.9246


# Задача регрессии

Разделение на признаки и целевую переменную.
Исключаем loan_status, так как это целевая переменная для другой задачи. Далее снова разделяем на обучающую и тестовые выборки.

In [ ]:
X_reg = df.drop(['credit_score', 'loan_status'], axis=1)
y_reg = df['credit_score']

X_reg_train, X_reg_test, y_reg_train, y_reg_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Определим числовых и категориальных столбцов (теперь признаков меньше). Создадим препроцессор и пайплаин для KNN регрессора.

In [ ]:
numerical_features_reg = X_reg.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features_reg = X_reg.select_dtypes(include=['object']).columns.tolist()

preprocessor_reg = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features_reg),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_reg)
    ])

baseline_knn_reg = Pipeline(steps=[
    ('preprocessor', preprocessor_reg),
    ('regressor', KNeighborsRegressor())
])

Снова обучим модель и оценим нашу модель.

In [ ]:
baseline_knn_reg.fit(X_reg_train, y_reg_train)

y_reg_pred = baseline_knn_reg.predict(X_reg_test)

print("\n=== БЕЙЗЛАЙН - РЕГРЕССИЯ ===")
print(f"RMSE: {np.sqrt(mean_squared_error(y_reg_test, y_reg_pred)):.4f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred):.4f}")
print(f"R²: {r2_score(y_reg_test, y_reg_pred):.4f}")


=== БЕЙЗЛАЙН - РЕГРЕССИЯ ===
RMSE: 52.8817
MAE: 42.2389
R²: -0.0693


Выводы:

Простейшие алгоритмы без каких-либо улучшени дают довольно слабые результаты. Хотя классификация все же показывает точность предсказаний 0.89, это необходимо повысить. Более того, для регрессии получены довольно большие ошибки, которые необходимо минимизировать дальнейшими улучшениями.

# 3. Улучшение бейзлайна

Как улучшить?

Формулировка гипотез

Препроцессинг: KNN чувствителен к масштабу данных. Наша стандартизация верна, но можно проверить RobustScaler для устойчивости к выбросам.

Визуализация и анализ данных: Выявление выбросов и аномалий в числовых признаках (например, person_age, person_income), которые могут "ломать" метрику расстояния. Их можно ограничить (capping).

Подбор гиперпараметров: Самый важный шаг для KNN. Подбор оптимального n_neighbors, weights (равные или по расстоянию), p (метрика Минковского).

Кросс-валидация: Использование GridSearchCV для надёжного подбора гиперпараметров, избегая переобучения.

Дисбаланс классов (для классификации): Использование class_weight='distance' или применение техник сэмплирования (SMOTE) внутри кросс-валидации.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_regression

Сначала улучшим классификацию

In [ ]:
improved_knn_clf = Pipeline(steps=[
    ('preprocessor', preprocessor_clf),
    ('classifier', KNeighborsClassifier())
])

Славним разные скейлеры, подберем гиперпараметры с учетом кросс-валидации.

In [ ]:
param_grid_clf = {
    'classifier__n_neighbors': range(3, 15, 2),
    'classifier__weights': ['uniform', 'distance'],
    'classifier__p': [1, 2],
    'preprocessor__num': [StandardScaler(), RobustScaler()]
}

grid_search_clf = GridSearchCV(improved_knn_clf, param_grid_clf, cv=5, scoring='f1', n_jobs=-1, verbose=1)
grid_search_clf.fit(X_clf_train, y_clf_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['person_age',
                                                                          'person_income',
                                                                          'person_emp_exp',
                                                                          'loan_amnt',
                                                                          'loan_int_rate',
                                                                          'loan_percent_income',
                                                                          'cb_person_cred_hist_length',
                                                                          'credit_score']),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['person_gender',
                                                                          'person_education',
                                                                          'person_home_ownership',
                                                                          'loan_intent',
                                                                          'previous_loan_defaults_on_file'])])),
                                       ('classifier', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'classifier__n_neighbors': range(3, 15, 2),
                         'classifier__p': [1, 2],
                         'classifier__weights': ['uniform', 'distance'],
                         'preprocessor__num': [StandardScaler(),
                                               RobustScaler()]},
             scoring='f1', verbose=1)

Предскажем новые результаты с помощью улучшенной модели и посмотрим значения метрик.

In [ ]:
best_knn_clf = grid_search_clf.best_estimator_

y_clf_pred_improved = best_knn_clf.predict(X_clf_test)
y_clf_pred_proba_improved = best_knn_clf.predict_proba(X_clf_test)[:, 1]

print("=== УЛУЧШЕННАЯ МОДЕЛЬ - КЛАССИФИКАЦИЯ ===")
print(f"Лучшие параметры: {grid_search_clf.best_params_}")
print(f"F1-Score: {f1_score(y_clf_test, y_clf_pred_improved):.4f}")
print(f"Accuracy: {accuracy_score(y_clf_test, y_clf_pred_improved):.4f}")
print(f"ROC-AUC: {roc_auc_score(y_clf_test, y_clf_pred_proba_improved):.4f}")

=== УЛУЧШЕННАЯ МОДЕЛЬ - КЛАССИФИКАЦИЯ ===
Лучшие параметры: {'classifier__n_neighbors': 13, 'classifier__p': 1, 'classifier__weights': 'distance', 'preprocessor__num': StandardScaler()}
F1-Score: 0.7678
Accuracy: 0.9041
ROC-AUC: 0.9533


Теперь улучшим регрессию. Выполняются аналогичные действия, поэтому не будем дублировать комментарии.

In [ ]:
improved_knn_reg = Pipeline(steps=[
    ('preprocessor', preprocessor_reg),
    ('regressor', KNeighborsRegressor())
])

param_grid_reg = {
    'regressor__n_neighbors': range(3, 15, 2),
    'regressor__weights': ['uniform', 'distance'],
    'regressor__p': [1, 2],
    'preprocessor__num': [StandardScaler(), RobustScaler()]
}

grid_search_reg = GridSearchCV(improved_knn_reg, param_grid_reg, cv=5, scoring='neg_root_mean_squared_error', n_jobs=-1, verbose=1)
grid_search_reg.fit(X_reg_train, y_reg_train)

best_knn_reg = grid_search_reg.best_estimator_

y_reg_pred_improved = best_knn_reg.predict(X_reg_test)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


In [ ]:
print("\n=== УЛУЧШЕННАЯ МОДЕЛЬ - РЕГРЕССИЯ ===")
print(f"Лучшие параметры: {grid_search_reg.best_params_}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_improved)):.4f}")
print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_improved):.4f}")
print(f"R²: {r2_score(y_reg_test, y_reg_pred_improved):.4f}")


=== УЛУЧШЕННАЯ МОДЕЛЬ - РЕГРЕССИЯ ===
Лучшие параметры: {'preprocessor__num': StandardScaler(), 'regressor__n_neighbors': 13, 'regressor__p': 2, 'regressor__weights': 'uniform'}
RMSE: 50.0911
MAE: 40.0385
R²: 0.0406


Подбор гиперпараметров является критически важным этапом для алгоритма KNN. Выбор метрики расстояния и весов соседей может кардинально изменить поведение модели. Препроцессинг данных также играет ключевую роль.

# Самостоятельная реализация KNN

In [ ]:
import numpy as np
from collections import Counter
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
class MyKNeighborsClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_neighbors=5, weights='uniform', p=2):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.p = p

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.X_ = X
        self.y_ = y
        self.classes_ = np.unique(y)
        return self

    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        y_pred = [self._predict_single(x) for x in X]
        return np.array(y_pred)

    def _predict_single(self, x):
        # Вычисляем расстояния до всех точек обучающей выборки
        distances = np.linalg.norm(self.X_ - x, ord=self.p, axis=1)

        # Получаем индексы k ближайших соседей
        k_indices = np.argpartition(distances, self.n_neighbors)[:self.n_neighbors]

        # Метки k ближайших соседей
        k_nearest_labels = self.y_[k_indices]

        if self.weights == 'uniform':
            # Простое голосование большинства
            most_common = Counter(k_nearest_labels).most_common(1)
            return most_common[0][0]
        else:
            # Взвешивание по обратному расстоянию
            weights = 1 / (distances[k_indices] + 1e-8)
            weighted_votes = {}
            for label, weight in zip(k_nearest_labels, weights):
                weighted_votes[label] = weighted_votes.get(label, 0) + weight
            return max(weighted_votes, key=weighted_votes.get)

In [ ]:
class MyKNeighborsRegressor(BaseEstimator, RegressorMixin):
    def __init__(self, n_neighbors=5, weights='uniform', p=2):
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.p = p

    def fit(self, X, y):
        X, y = check_X_y(X, y)
        self.X_ = X
        self.y_ = y
        return self

    def predict(self, X):
        check_is_fitted(self)
        X = check_array(X)
        y_pred = [self._predict_single(x) for x in X]
        return np.array(y_pred)

    def _predict_single(self, x):
        distances = np.linalg.norm(self.X_ - x, ord=self.p, axis=1)
        k_indices = np.argpartition(distances, self.n_neighbors)[:self.n_neighbors]
        k_nearest_labels = self.y_[k_indices]

        if self.weights == 'uniform':
            return np.mean(k_nearest_labels)
        else:
            weights = 1 / (distances[k_indices] + 1e-8)
            return np.average(k_nearest_labels, weights=weights)

Обучение, оценка и сравнение своей реализации с предыдущими реализациями

In [ ]:
# Препроцессинг для своей реализации
preprocessor_clf.fit(X_clf_train)
X_clf_train_processed = preprocessor_clf.transform(X_clf_train)
X_clf_test_processed = preprocessor_clf.transform(X_clf_test)

# Обучение своей модели KNN для классификации
my_knn_clf = MyKNeighborsClassifier(n_neighbors=5)
my_knn_clf.fit(X_clf_train_processed, y_clf_train)
y_clf_pred_my = my_knn_clf.predict(X_clf_test_processed)

print("=== МОЯ РЕАЛИЗАЦИЯ - КЛАССИФИКАЦИЯ ===")
print(f"F1-Score: {f1_score(y_clf_test, y_clf_pred_my):.4f}")

=== МОЯ РЕАЛИЗАЦИЯ - КЛАССИФИКАЦИЯ ===
F1-Score: 0.7541


Если будем сравнивать с изначальным бейзлайном, можем заметить, что результаты получились очень похожими.

```
=== БЕЙЗЛАЙН - КЛАССИФИКАЦИЯ ===
F1-Score: 0.7541
```

Моя реализация

```
=== МОЯ РЕАЛИЗАЦИЯ - КЛАССИФИКАЦИЯ ===
F1-Score: 0.7541
```

Проведем аналогичные действия с регрессией

In [ ]:
preprocessor_reg.fit(X_reg_train)
X_reg_train_processed = preprocessor_reg.transform(X_reg_train)
X_reg_test_processed = preprocessor_reg.transform(X_reg_test)

my_knn_reg = MyKNeighborsRegressor(n_neighbors=5)
my_knn_reg.fit(X_reg_train_processed, y_reg_train)
y_reg_pred_my = my_knn_reg.predict(X_reg_test_processed)

print("\n=== МОЯ РЕАЛИЗАЦИЯ - РЕГРЕССИЯ ===")
print(f"RMSE: {np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my)):.4f}")


=== МОЯ РЕАЛИЗАЦИЯ - РЕГРЕССИЯ ===
RMSE: 52.8817


Теперь сравним регрессию

```
=== БЕЙЗЛАЙН - РЕГРЕССИЯ ===
RMSE: 52.8817
```

Моя реализация

```
=== МОЯ РЕАЛИЗАЦИЯ - РЕГРЕССИЯ ===
RMSE: 52.8817
```
Наблюдаем аналогичную ситуацию, результаты получились похожими.

Используем лучшие параметры для классификации, найденные ранее

In [ ]:
best_params_clf = grid_search_clf.best_params_

my_knn_clf_improved = MyKNeighborsClassifier(
    n_neighbors=best_params_clf['classifier__n_neighbors'],
    weights=best_params_clf['classifier__weights'],
    p=best_params_clf['classifier__p']
)
my_knn_clf_improved.fit(X_clf_train_processed, y_clf_train)
y_clf_pred_my_improved = my_knn_clf_improved.predict(X_clf_test_processed)

print("=== МОЯ УЛУЧШЕННАЯ РЕАЛИЗАЦИЯ - КЛАССИФИКАЦИЯ ===")
print(f"F1-Score: {f1_score(y_clf_test, y_clf_pred_my_improved):.4f}")

=== МОЯ УЛУЧШЕННАЯ РЕАЛИЗАЦИЯ - КЛАССИФИКАЦИЯ ===
F1-Score: 0.7678


Используем лучшие параметры для регрессии, найденные ранее. Создаем улучшенную версию нашего KNN регрессора с лучшими параметрами. Потом обучаем модель, делаем предсказания и оцениваем нашу модель.

In [ ]:
best_params_reg = grid_search_reg.best_params_
print(f"Лучшие параметры для регрессии: {best_params_reg}")

my_knn_reg_improved = MyKNeighborsRegressor(
    n_neighbors=best_params_reg['regressor__n_neighbors'],
    weights=best_params_reg['regressor__weights'],
    p=best_params_reg['regressor__p']
)

my_knn_reg_improved.fit(X_reg_train_processed, y_reg_train)

y_reg_pred_my_improved = my_knn_reg_improved.predict(X_reg_test_processed)

print("=== МОЯ УЛУЧШЕННАЯ РЕАЛИЗАЦИЯ - РЕГРЕССИЯ ===")
print(f"RMSE: {np.sqrt(mean_squared_error(y_reg_test, y_reg_pred_my_improved)):.4f}")
# print(f"MAE: {mean_absolute_error(y_reg_test, y_reg_pred_my_improved):.4f}")
# print(f"R²: {r2_score(y_reg_test, y_reg_pred_my_improved):.4f}")

Лучшие параметры для регрессии: {'preprocessor__num': StandardScaler(), 'regressor__n_neighbors': 13, 'regressor__p': 2, 'regressor__weights': 'uniform'}
=== МОЯ УЛУЧШЕННАЯ РЕАЛИЗАЦИЯ - РЕГРЕССИЯ ===
RMSE: 50.0911


**Финальные выводы**

- Моя реализация KNN показала результаты, сопоставимые с sklearn при одинаковых параметрах, что подтверждает её корректность.

- Применение техник улучшения бейзлайна (подбор гиперпараметров) к моей реализации позволило достичь такого же значительного прироста качества, как и у модели из sklearn.

- Это доказывает, что успех в машинном обучении часто зависит не от "магии" библиотек, а от корректной реализации алгоритмов и, что более важно, от тщательной настройки и предобработки данных.

- KNN — простой для понимания и реализации алгоритм, но он может быть очень эффективным на небольших и средних наборах данных при правильной настройке.